# File for defining Similarity
### Things we might want: high reviews by same person, same tags...

In [208]:
import pandas as pd
import numpy
import random
import torch
from torch.nn import functional as F
import numpy as np
import collections

%run Training.ipynb

torch.cuda.is_available()

True

In [645]:
r = pd.read_csv( 'ratings.csv' )
tr = pd.read_csv('to_read.csv')
b = pd.read_csv('books.csv')
t = pd.read_csv('tags.csv')
bt = pd.read_csv('book_tags.csv')


In [646]:
bt = bt.merge(b[['goodreads_book_id','title']], on = 'goodreads_book_id')
bt.loc[bt['count'] < 0, 'count'] = 0
bt

,goodreads_book_id,tag_id,count,title
0,1,30574,167697,Harry Potter and the Half-Blood Prince (Harry ...
1,1,11305,37174,Harry Potter and the Half-Blood Prince (Harry ...
2,1,11557,34173,Harry Potter and the Half-Blood Prince (Harry ...
3,1,8717,12986,Harry Potter and the Half-Blood Prince (Harry ...
4,1,33114,12716,Harry Potter and the Half-Blood Prince (Harry ...
...,...,...,...,...
999907,33288638,21303,7,Wait for It
999908,33288638,17271,7,Wait for It
999909,33288638,1126,7,Wait for It
999910,33288638,11478,7,Wait for It


In [6]:
r_high = r[r['rating']>3]
r_high_by_user = r_high.sort_values('user_id')

In [19]:
len(r_high_by_user)

4122111

In [7]:
r_high_by_user.loc[0]

user_id      1
book_id    258
rating       5
Name: 0, dtype: int64

In [36]:
r_high_by_user

,user_id,book_id,rating
0,1,258,5
1936401,1,2770,4
1936403,1,198,4
1936419,1,1761,4
1936422,1,128,5
...,...,...,...
5404296,53424,9255,4
5404364,53424,43,5
5404333,53424,17,4
5404301,53424,4645,5


In [35]:
list(zip(r_high_by_user.user_id,r_high_by_user.book_id))[1]

(1, 2770)

In [33]:
r_high_by_user.loc[1]

user_id       2
book_id    4081
rating        4
Name: 1, dtype: int64

In [28]:
user_book_dict = dict.fromkeys(r_high_by_user.user_id.unique(),[])
len(user_book_dict)

53406

In [757]:
r_high_by_user.loc[0]

user_id      1
book_id    258
rating       5
Name: 0, dtype: int64

In [751]:
r_high_by_user.columns

Index(['book_id'], dtype='object')

In [196]:
r_high_by_user = r_high_by_user[['user_id','book_id']]
r_high_by_user

,user_id,book_id
0,1,258
1936401,1,2770
1936403,1,198
1936419,1,1761
1936422,1,128
...,...,...
5404296,53424,9255
5404364,53424,43
5404333,53424,17
5404301,53424,4645


In [123]:
# r_high_by_user['book_id'] = r_high_by_user['book_id'].apply(lambda x: [x])

In [239]:
r_high_by_user.head(65)

,book_id
user_id,
1,258
1,2770
1,198
1,1761
1,128
...,...
1,901
2,27
2,9762


In [197]:
r_high_by_user.set_index('user_id',inplace=True)

In [312]:
r_high_by_user.book_id.sample(5).values[0]

193

In [307]:
r_high_by_user.book_id.value_counts().describe()

count    10000.000000
mean       412.211100
std        921.972938
min          4.000000
25%        103.000000
50%        170.000000
75%        347.000000
max      18926.000000
Name: book_id, dtype: float64

In [43]:
a = dict.fromkeys(['a','b','c'],[])
list(a.keys())


['a', 'b', 'c']

In [611]:
r_high_by_user

,user_id,book_id,rating
0,1,258,5
1936401,1,2770,4
1936403,1,198,4
1936419,1,1761,4
1936422,1,128,5
...,...,...,...
5404296,53424,9255,4
5404364,53424,43,5
5404333,53424,17,4
5404301,53424,4645,5


In [612]:
class dictGen():
    def __init__(self,num_classes, batch_size,df,keys,values):
        self.dict = self.__init_dict(df,keys,values)
        self.keys = list(self.dict.keys())
        self.num_classes = num_classes
        self.batch_size = batch_size
        
        #iter vars
        self.idx = 0
        
    def __next__(self):
        if self.idx + self.batch_size > len(self.keys):
            self.idx = 0
            random.shuffle(self.keys)
            
        X, Y = [],[]
        
        for i in range(self.batch_size):
            curr_user_idx = self.keys[self.idx]
            curr_user_book_list = self.dict[curr_user_idx]
            x,y = random.sample(curr_user_book_list,2)
            
            X.append(x)
            Y.append(y)
            self.idx +=1
            
        X = F.one_hot(torch.tensor(X)-1, self.num_classes).float()
        Y = F.one_hot(torch.tensor(Y)-1, self.num_classes).float()
        
        return X.cuda(),Y.cuda()
    
    
    def __iter__(self):
        pass
        
    
    def __init_dict(self,df,keys,values):
        _list = list(zip(df[keys],df[values]))
        c = collections.defaultdict(list)
        for a,b in _list:
            c[a].extend([b])
            
        for key in list(c.keys()):
            if len(c[key]) < 2:
                del c[key]
        return c

In [313]:
len([1]) == len([1]) ==len([1])


True

In [635]:
class NegativeSampleGen():
    def __init__(self,num_classes, batch_size, num_neg_samples, df,keys,values):
        self.df = df
        self.keys = keys
        self.values = values
        self.dict = self.__init_dict(df,keys,values)
        self.keys = list(self.dict.keys())
        self.num_classes = num_classes
        self.batch_size = batch_size
        self.num_neg_samples = num_neg_samples
        
        self.idx = 0
        
    def __next__(self):
        if self.idx + self.batch_size > len(self.keys):
            self.idx = 0
            random.shuffle(self.keys)
            
        context_list, X_list, labels = [], [], []
        
        for i in range(self.batch_size):
            #print(i,labels)
            curr_user_idx = self.keys[self.idx]
            curr_user_book_list = self.dict[curr_user_idx]
            context, positive = random.sample(curr_user_book_list, 2)
            
            contexts = [context for _ in range(self.num_neg_samples+1)]
                        
            negative_samples = list(self.df[self.values].sample(self.num_neg_samples).values)
            
            prop_list = [positive] + negative_samples
            
            label_list = [1] + [1 for _ in range(self.num_neg_samples)]
            
            assert(len(prop_list)==len(label_list)==len(contexts))
            
            context_list += (contexts)
            X_list += (prop_list)
            labels += (label_list)
            
            self.idx +=1
        
        assert(len(context_list)==len(X_list)==len(labels))
        
#         zipped = list(zip(context_list,X_list, labels))
#         random.shuffle(zipped)
        
#         contexts, X, labels = zip(*zipped)
                        
        #contexts = F.one_hot(torch.tensor(context_list), self.num_classes).float()
        X = F.one_hot(torch.tensor(X_list), self.num_classes).float()
        contexts = torch.ones((X.shape))
        labels = torch.tensor(labels).unsqueeze(dim=1).float()
        
        return contexts.cuda(), X.cuda(), labels.cuda()
    
    
    def __iter__(self):
        pass
        
    def __get_dist(self,df,value):
        pass
        
    def __init_dict(self,df,keys,values):
        _list = list(zip(df[keys],df[values]))
        c = collections.defaultdict(list)
        for a,b in _list:
            c[a].extend([b])
            
        for key in list(c.keys()):
            if len(c[key]) < 2:
                del c[key]
        print(len(c),'c length')
        return c

In [636]:
dicGen = NegativeSampleGen(10001,4,4,r_high_by_user.head(300),'user_id', 'book_id')

6 c length


In [637]:
dicGen.__next__()[2].shape


torch.Size([20, 1])

In [638]:
%run Training.ipynb
trainer = ContrastiveTrainer(3e-3,10001,300,0,25,'contrastive','./overfit9',dicGen)
trainer.train(5,40000)
#trainer.model((z,y))


yes


KeyboardInterrupt: 

In [ ]:
context, X, _ = dicGen.__next__()
twinNetwork((context,X))

In [261]:
dicGen = dictGen(10000,256,r_high_by_user,'user_id', 'book_id')